In [1]:
from torchvision.models import ResNet50_Weights, resnet50
import os
import torch

def print_model_size(mdl):
    torch.save(mdl.state_dict(), "tmp.pt")
    print("%.2f MB" %(os.path.getsize("tmp.pt")/1e6))
    os.remove('tmp.pt')

/home/aiteam/miniconda3/envs/loader/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
standard_model = resnet50(weights=ResNet50_Weights.auto)
standard_model.eval()
backend = 'fbgemm' # "qnnpack" 
standard_model.qconfig = torch.quantization.get_default_qconfig(backend)
torch.backends.quantized.engine = backend
model_static_quantized = torch.quantization.prepare(standard_model, inplace=False)
model_static_quantized = torch.quantization.convert(model_static_quantized, inplace=False)

print_model_size(standard_model)
print_model_size(model_static_quantized)

/home/aiteam/miniconda3/envs/loader/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/aiteam/miniconda3/envs/loader/lib/python3.9/site-packages/torch/ao/quantization/observer.py:176: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/home/aiteam/miniconda3/envs/loader/lib/python3.9/site-packages/torch/ao/quantization/observer.py:1135: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


102.54 MB
26.57 MB


In [7]:
%%timeit -r 10 -n 3
input_fp32 = torch.randn(32, 3, 224, 224)
res = standard_model(input_fp32)

693 ms ± 61.3 ms per loop (mean ± std. dev. of 10 runs, 3 loops each)


In [ ]:
%%timeit -r 10 -n 3
input_fp32 = torch.randn(32, 3, 224, 224)
res = model_static_quantized(input_fp32)
# 그냥 돌리면 에러 남

In [15]:
class M(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.quant = torch.quantization.QuantStub()
        self.model = model
        self.dequant = torch.quantization.DeQuantStub()
    def forward(self, x):
        # during the convert step, this will be replaced with a
        # `quantize_per_tensor` call
        x = self.quant(x)
        x = self.model(x)
        x = self.dequant(x)
        return x

In [18]:
%%capture
standard_model = resnet50(weights=ResNet50_Weights.auto)
model_fp32 = M(standard_model)
model_fp32.eval()

In [19]:
backend = "qnnpack"
model_fp32.qconfig = torch.quantization.get_default_qconfig(backend)
model_fp32_prepared = torch.quantization.prepare(model_fp32, inplace=False)
model_static_quantized = torch.quantization.convert(model_fp32_prepared, inplace=False)

/home/aiteam/miniconda3/envs/loader/lib/python3.9/site-packages/torch/ao/quantization/observer.py:1135: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


In [20]:
%%timeit -r 10 -n 3
input_fp32 = torch.randn(32, 3, 224, 224)
res = model_static_quantized(input_fp32)

NotImplementedError: Could not run 'aten::add.out' with arguments from the 'QuantizedCPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::add.out' is only available for these backends: [Dense, FPGA, Conjugate, VmapMode, FuncTorchGradWrapper, Functionalize, MPS, IPU, UNKNOWN_TENSOR_TYPE_ID, QuantizedXPU, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, SparseCPU, SparseCUDA, SparseHIP, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, SparseXPU, UNKNOWN_TENSOR_TYPE_ID, SparseVE, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, NestedTensorCUDA, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID,

In [11]:

backend = "qnnpack"
standard_model.qconfig = torch.quantization.get_default_qconfig(backend)
torch.backends.quantized.engine = backend

standard_model.eval()
model_fp32_fused = torch.quantization.fuse_modules(standard_model, [['conv1', 'bn1']])

model_fp32_prepared = torch.quantization.prepare(model_fp32_fused, inplace=False)

# calibration
input_fp32 = torch.randn(4, 1, 4, 4)
model_fp32_prepared(input_fp32)

model_static_quantized = torch.quantization.convert(model_fp32_prepared, inplace=False)

AssertionError: Fusion only for eval!

In [ ]:
print_model_size(model_static_quantized)